In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from RandomForestForTimeSeriesClassifier import RandomForestForTimeSeriesClassifier

In [2]:
dataset = pd.read_csv('../datasets/sp_500_stocks/sp500_stocks.csv', nrows=40000)

In [3]:
dataset = dataset.dropna()
dataset

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
22524,2010-01-04,AMD,9.700000,9.700000,9.900000,9.680000,9.790000,18748700.0
22525,2010-01-05,AMD,9.710000,9.710000,9.900000,9.680000,9.710000,22145700.0
22526,2010-01-06,AMD,9.570000,9.570000,9.760000,9.550000,9.680000,18643400.0
22527,2010-01-07,AMD,9.470000,9.470000,9.550000,9.180000,9.510000,26806800.0
22528,2010-01-08,AMD,9.430000,9.430000,9.470000,9.290000,9.370000,13752800.0
...,...,...,...,...,...,...,...,...
39995,2019-10-04,APD,190.052368,213.880005,214.899994,211.580002,212.130005,770700.0
39996,2019-10-07,APD,190.123444,213.960007,215.899994,212.500000,213.330002,727800.0
39997,2019-10-08,APD,184.614120,207.759995,213.350006,207.660004,213.350006,1025300.0
39998,2019-10-09,APD,188.150772,211.740005,212.660004,208.820007,210.119995,810100.0


In [4]:
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset['Date'] = dataset['Date'].map(dt.datetime.toordinal)
#dataset['Date'] = pd.to_float(dataset['Date'])
# dataset['Date'] = datetime.datetime.strptime(dataset['Date'], '%Y-%m-%d')
# dataset['Date'] = float(time.mktime(dataset['Date']))
dataset['Symbol'] = pd.factorize(dataset.Symbol)[0]
dataset['Adj Close'] = dataset['Adj Close'] * 100
dataset['Adj Close'] = np.floor(pd.to_numeric(dataset['Adj Close'], errors='coerce')).astype('Int64')
dataset['Close'] = dataset['Close'] * 100
dataset['Close'] = np.floor(pd.to_numeric(dataset['Adj Close'], errors='coerce')).astype('Int64')
dataset['High'] = dataset['High'] * 100
dataset['High'] = np.floor(pd.to_numeric(dataset['Adj Close'], errors='coerce')).astype('Int64')
dataset['Low'] = dataset['Low'] * 100
dataset['Low'] = np.floor(pd.to_numeric(dataset['Adj Close'], errors='coerce')).astype('Int64')
dataset['Open'] = dataset['Open'] * 100
dataset['Open'] = np.floor(pd.to_numeric(dataset['Adj Close'], errors='coerce')).astype('Int64')
dataset['Volume'] = np.floor(pd.to_numeric(dataset['Volume'], errors='coerce')).astype('Int64')
dataset

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
22524,733776,0,969,969,969,969,969,18748700
22525,733777,0,971,971,971,971,971,22145700
22526,733778,0,956,956,956,956,956,18643400
22527,733779,0,947,947,947,947,947,26806800
22528,733780,0,943,943,943,943,943,13752800
...,...,...,...,...,...,...,...,...
39995,737336,1,19005,19005,19005,19005,19005,770700
39996,737339,1,19012,19012,19012,19012,19012,727800
39997,737340,1,18461,18461,18461,18461,18461,1025300
39998,737341,1,18815,18815,18815,18815,18815,810100


In [5]:
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(dataset)
# normalized_dataset = pd.DataFrame(x_scaled)
#
# normalized_dataset

In [6]:
# X = dataset.iloc[:, 1:].values
#
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

aux = dataset.copy()

X = aux.pop("Date")

# Obtener los valores de la variable dependiente (objetivo)
Y = aux.copy()
dataset = aux

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=1)

X_train = X_train.values.reshape(-1, 1)
X_test = X_test.values.reshape(-1, 1)


In [7]:
basicRF = RandomForestClassifier(n_jobs=1, n_estimators=20)

In [8]:
basicRF.fit(X_train, Y_train)

RandomForestClassifier(n_estimators=20, n_jobs=1)

In [9]:
# Realizar predicciones sobre los datos de prueba
y_pred = basicRF.predict(X_test)

In [10]:
from sklearn import metrics

print("R2:")
print(r2_score(Y_test,y_pred))
print(np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))

R2:
-1.2041371781289059
15766703.306495855


In [11]:
rfts = RandomForestForTimeSeriesClassifier(n_jobs=1, n_estimators=20)

In [12]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [13]:
rfts.fit(X_train, Y_train)

RandomForestForTimeSeriesClassifier(n_estimators=20, n_jobs=1)

In [14]:
y_pred = rfts.predict(X_test)

In [15]:
print("R2:")
print(r2_score(Y_test,y_pred))
print(np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))

R2:
-1.0442682888845416
13664554.94575378
